In [1]:
# 引入rich打印
import sys
sys.path.append('../..')

from rich import print
from llm_common.set_env import set_open_ai_key,print_current_path

set_open_ai_key()
print_current_path()

7RGC8kgFXY1IT9-vxITDvXolfh3-8A

当前目录: e:\github_project\robin-learn-llm\lang_chain\c01_prompts

In [2]:
# 提示词模板
from langchain_core.prompts import PromptTemplate

template = "Translate the following text to {language}: {text}"
prompt_template = PromptTemplate(template=template)

prompt = prompt_template.format(language="French", text="Hello, how are you?")
print(prompt)

Translate the following text to French: Hello, how are you?

In [3]:
# 聊天提示词模板
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

print(prompt_value)

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a helpful AI bot. Your name is Bob.',
            additional_kwargs={},
            response_metadata={}
        ),
        HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
        AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}),
        HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})
    ]
)

In [4]:
# 少样本提示词模板 - 1.构造样本
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
]

In [5]:
# 少样本提示词模板 - 2.构造样本提示词模板
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=[]
)

print(few_shot_prompt.invoke({}))

ChatPromptValue(
    messages=[
        HumanMessage(content='2 🦜 2', additional_kwargs={}, response_metadata={}),
        AIMessage(content='4', additional_kwargs={}, response_metadata={}),
        HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}),
        AIMessage(content='5', additional_kwargs={}, response_metadata={})
    ]
)

In [6]:
# 少样本提示词模板 - 3.使用样本提示词模板构造聊天提示词模板
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt)

ChatPromptTemplate(
    input_variables=['input'],
    input_types={},
    partial_variables={},
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='You are a wondrous wizard of math.'
            ),
            additional_kwargs={}
        ),
        FewShotChatMessagePromptTemplate(
            examples=[{'input': '2 🦜 2', 'output': '4'}, {'input': '2 🦜 3', 'output': '5'}],
            input_variables=[],
            input_types={},
            partial_variables={},
            example_prompt=ChatPromptTemplate(
                input_variables=['input', 'output'],
                input_types={},
                partial_variables={},
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['input'],
                            input_types={},
                            partial_variables={},
                            template='{input}'
                        ),
                        additional_kwargs={}
                    ),
                    AIMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['output'],
                            input_types={},
                            partial_variables={},
                            template='{output}'
                        ),
                        additional_kwargs={}
                    )
                ]
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['input'],
                input_types={},
                partial_variables={},
                template='{input}'
            ),
            additional_kwargs={}
        )
    ]
)

In [7]:
# 少样本提示词模板 - 4. 使用少样本提示词，调用模型
from langchain_openai import ChatOpenAI

llm  = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

chain = final_prompt | llm

result = chain.invoke({"input": "what is 28 🦜 12" })

print(result)

AIMessage(
    content='28 🦜 12 equals 40.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 10,
            'prompt_tokens': 59,
            'total_tokens': 69,
            'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0},
            'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}
        },
        'model_name': 'gpt-4o-mini-2024-07-18',
        'system_fingerprint': 'fp_f85bea6784',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-897ef880-908f-45c8-93ca-6086e89a3596-0',
    usage_metadata={
        'input_tokens': 59,
        'output_tokens': 10,
        'total_tokens': 69,
        'input_token_details': {'cache_read': 0},
        'output_token_details': {'reasoning': 0}
    }
)